<a href="https://colab.research.google.com/github/rosafilgueira/PyCodeSearch/blob/main/Registry_search_multimodel_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Transformer models for code


## UnixCoders Models

We are going to use two models:
-  Lazyhope/unixcoder-nine-advtest, wich is used by the original RepoSim pipeline.

- Lazyhope/unixcoder-clone-detection

Both have been trained following: https://github.com/microsoft/CodeBERT/blob/master/UniXcoder/downstream-tasks/code-search/README.md

We are going to work with these two models (which uses a bi-enconder approach) for code similarity and text similarity.

In [1]:
!pip install tensorflow
!pip install -U accelerate
!pip install docarray
!pip install pandas
!pip install torch
!pip install transformers
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.2/263.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.0 MB/s eta 0:00:00


## Add database example

In [14]:
import pandas as pd

# Define the codes and their corresponding docstrings
codes = [
    "def add(a, b):\n    return a + b",
    "def subtract(a, b):\n    return a - b",
    "def multiply(a, b):\n    return a * b",
    "def divide(a, b):\n    return a / b",
    "def power(a, b):\n    return a ** b",
    "def modulus(a, b):\n    return a % b"
]

docs = [
    "This function adds two numbers.",
    "This function subtracts the second number from the first.",
    "This function multiplies two numbers.",
    "This function divides the first number by the second.",
    "This function raises the first number to the power of the second.",
    "This function returns the remainder when the first number is divided by the second."
]

# Create the dataframe
registry = pd.DataFrame({
    'code': codes,
    'doc': docs
})

### dataframe
registry

,code,doc
0,"def add(a, b):\n return a + b",This function adds two numbers.
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...
2,"def multiply(a, b):\n return a * b",This function multiplies two numbers.
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...
4,"def power(a, b):\n return a ** b",This function raises the first number to the p...
5,"def modulus(a, b):\n return a % b",This function returns the remainder when the f...


## Load two models for two different tasks

- model_code_to_code -- for code-to-code search
- model_text_to_code -- for text-to-code search
- model_bm25 - for code completion

In [3]:
from transformers import pipeline,  AutoModel, AutoTokenizer

model_code_to_code = pipeline(
    model="Lazyhope/unixcoder-clone-detection",
    trust_remote_code=True,
    device_map="auto")

## Note: this pipeline uses "Lazyhope/unixcoder-nine-advtest" underneed
model_text_to_code = pipeline(
    model="Lazyhope/RepoSim",
    trust_remote_code=True,
    device_map="auto")

model_name_1= "Luyu/bert-base-mdoc-bm25"
model_code_completion_1 = AutoModel.from_pretrained(model_name_1)
tokenizer_1 = AutoTokenizer.from_pretrained(model_name_1)

model_name_2= "microsoft/reacc-py-retriever"
model_code_completion_2 = AutoModel.from_pretrained(model_name_2)
tokenizer_2 = AutoTokenizer.from_pretrained(model_name_2)


A new version of the following files was downloaded from https://huggingface.co/Lazyhope/RepoSim:
- pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


[*] Consider setting GitHub token to avoid hitting rate limits. 
For more info, see: https://docs.github.com/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token


Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/reacc-py-retriever and are newly initialized: ['pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Turn code and docstring into torch embeddings

In [15]:
import torch

def encode(string, model_type):
    if model_type == 1:
        with torch.no_grad():
            embedding = model_text_to_code.encode(string, 512)
        final_t=embedding.squeeze()
    elif model_type == 2:
        with torch.no_grad():
            embedding = model_code_to_code(string, truncation=True, max_length=512)
        if isinstance(embedding, list):
            embedding = torch.tensor(embedding)
        kk=embedding.squeeze()
        final_t=kk[0]
    elif model_type ==3:
        with torch.no_grad():
            inputs = tokenizer_1(string, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
            outputs = model_code_completion_1(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1)  # You can use any pooling strategy here
        final_t = embedding.squeeze()

    else:
         with torch.no_grad():
            inputs = tokenizer_2(string, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
            outputs = model_code_completion_2(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1)  # You can use any pooling strategy here
         final_t = embedding.squeeze()

    return final_t

## model_type =1 -- for text-to_code
registry["doc_embeddings"] = registry["doc"].apply(encode, model_type=1)
## model_type=2 -- for code-to-code
registry["code_embeddings"] = registry["code"].apply(encode, model_type=1)

## model_type=3 | type=4 -- for code-completion
registry["code_complete_embeddings"] = registry["code"].apply(encode, model_type=3)


registry

,code,doc,doc_embeddings,code_embeddings,code_complete_embeddings
0,"def add(a, b):\n return a + b",This function adds two numbers.,"[tensor(-1.5903), tensor(-0.3973), tensor(3.99...","[tensor(-0.4317), tensor(-0.0293), tensor(2.66...","[tensor(0.3194), tensor(-0.1994), tensor(0.234..."
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...,"[tensor(-2.4678), tensor(-2.2801), tensor(0.66...","[tensor(-1.2667), tensor(-0.6932), tensor(-0.4...","[tensor(0.2588), tensor(-0.2688), tensor(0.324..."
2,"def multiply(a, b):\n return a * b",This function multiplies two numbers.,"[tensor(-2.3068), tensor(1.3498), tensor(3.078...","[tensor(-1.0338), tensor(1.4185), tensor(1.093...","[tensor(0.2858), tensor(-0.2316), tensor(0.240..."
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...,"[tensor(-3.0391), tensor(-0.0748), tensor(1.79...","[tensor(-0.9333), tensor(2.0973), tensor(0.789...","[tensor(0.3486), tensor(-0.2342), tensor(0.236..."
4,"def power(a, b):\n return a ** b",This function raises the first number to the p...,"[tensor(-2.9461), tensor(-0.5175), tensor(2.37...","[tensor(-2.0816), tensor(1.8068), tensor(0.650...","[tensor(0.4351), tensor(-0.2007), tensor(0.289..."
5,"def modulus(a, b):\n return a % b",This function returns the remainder when the f...,"[tensor(-2.4903), tensor(-1.3584), tensor(1.60...","[tensor(-0.2191), tensor(0.7266), tensor(-0.20...","[tensor(0.2112), tensor(-0.1506), tensor(0.531..."


### Text-to-code Similarity

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [6]:
# Compute user query embeddings
user_query_docs = "Function that adds two numbers"
#only change -- indicate the model_type=1 ... for tex-to-code
user_query_docs_emb = encode(user_query_docs, model_type=1)


# Convert document embeddings to numpy arrays
registry["doc_embeddings"] = registry["doc_embeddings"].apply(lambda x: np.array(x))

# Compute cosine similarity
user_query_emb = np.array(user_query_docs_emb)
cos_similarities = cosine_similarity(user_query_emb.reshape(1, -1), np.vstack(registry["doc_embeddings"]))

# Add cosine similarity scores as a new column
registry_doc=registry.copy()
registry_doc["cosine_similarity_doc"] = cos_similarities[0]

# Sort the dataframe based on cosine similarity
sorted_df = registry_doc.sort_values(by="cosine_similarity_doc", ascending=False)

# Retrieve the top 5 most similar documents
top_5_similar_docs = sorted_df.head(5)

In [9]:
top_5_similar_docs

,code,doc,doc_embeddings,code_embeddings,code_complete_embeddings,cosine_similarity_doc
0,"def add(a, b):\n return a + b",This function adds two numbers.,"[-1.590325, -0.39731324, 3.997744, 2.6590736, ...","[tensor(-0.4317), tensor(-0.0293), tensor(2.66...","[tensor(0.3194), tensor(-0.1994), tensor(0.234...",0.976803
2,"def multiply(a, b):\n return a * b",This function multiplies two numbers.,"[-2.3068378, 1.3498034, 3.0785217, 1.3164998, ...","[tensor(-1.0338), tensor(1.4185), tensor(1.093...","[tensor(0.2858), tensor(-0.2316), tensor(0.240...",0.704091
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...,"[-2.4677699, -2.280132, 0.6681174, 1.4311584, ...","[tensor(-1.2667), tensor(-0.6932), tensor(-0.4...","[tensor(0.2588), tensor(-0.2688), tensor(0.324...",0.700667
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...,"[-3.0390859, -0.07484988, 1.7925602, 0.2812554...","[tensor(-0.9333), tensor(2.0973), tensor(0.789...","[tensor(0.3486), tensor(-0.2342), tensor(0.236...",0.579780
5,"def modulus(a, b):\n return a % b",This function returns the remainder when the f...,"[-2.490343, -1.3584495, 1.6064605, -0.03004119...","[tensor(-0.2191), tensor(0.7266), tensor(-0.20...","[tensor(0.2112), tensor(-0.1506), tensor(0.531...",0.545701


### Code-completion

In [16]:
# Compute user query embeddings
user_query_code = "def add_numbers(a, b):\n return a "
#only change -- indicate the model_type=1 for advest, model_type= 2 for clone detection, model_type=3 for BM25, model_type 4= REACC
user_query_code_emb = encode(user_query_code, model_type=3)
# Convert document embeddings to numpy arrays
registry["code_complete_embeddings"] = registry["code_complete_embeddings"].apply(lambda x: np.array(x))

# Compute cosine similarity
user_query_emb_c = np.array(user_query_code_emb)
cos_similarities = cosine_similarity(user_query_emb_c.reshape(1, -1), np.vstack(registry["code_complete_embeddings"]))

# Add cosine similarity scores as a new column
registry_code=registry.copy()
registry_code["cosine_similarity_code_1"] = cos_similarities[0]

# Sort the dataframe based on cosine similarity
sorted_df_code = registry_code.sort_values(by="cosine_similarity_code_1", ascending=False)

# Retrieve the top 5 most similar documents
top_5_similar_code = sorted_df_code.head(5)
top_5_similar_code


,code,doc,doc_embeddings,code_embeddings,code_complete_embeddings,cosine_similarity_code_1
0,"def add(a, b):\n return a + b",This function adds two numbers.,"[tensor(-1.5903), tensor(-0.3973), tensor(3.99...","[tensor(-0.4317), tensor(-0.0293), tensor(2.66...","[0.31938395, -0.19940263, 0.23405677, -0.42716...",0.972584
2,"def multiply(a, b):\n return a * b",This function multiplies two numbers.,"[tensor(-2.3068), tensor(1.3498), tensor(3.078...","[tensor(-1.0338), tensor(1.4185), tensor(1.093...","[0.28581232, -0.23162104, 0.2407104, -0.225170...",0.968535
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...,"[tensor(-2.4678), tensor(-2.2801), tensor(0.66...","[tensor(-1.2667), tensor(-0.6932), tensor(-0.4...","[0.2588312, -0.2687941, 0.3244955, 0.07137667,...",0.943508
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...,"[tensor(-3.0391), tensor(-0.0748), tensor(1.79...","[tensor(-0.9333), tensor(2.0973), tensor(0.789...","[0.34864905, -0.23415264, 0.2369114, 0.0368931...",0.920648
4,"def power(a, b):\n return a ** b",This function raises the first number to the p...,"[tensor(-2.9461), tensor(-0.5175), tensor(2.37...","[tensor(-2.0816), tensor(1.8068), tensor(0.650...","[0.43510747, -0.20073725, 0.2897749, 0.0957916...",0.919372


In [17]:
# Compute user query embeddings
user_query_code = "def add_numbers(a, b):\n return a "
#only change -- indicate the model_type=1 for advest, model_type= 2 for clone detection, model_type=3 for BM25, model_type 4= REACC
user_query_code_emb = encode(user_query_code, model_type=1)
# Convert document embeddings to numpy arrays
registry["code_embeddings"] = registry["code_embeddings"].apply(lambda x: np.array(x))

# Compute cosine similarity
user_query_emb_c = np.array(user_query_code_emb)
cos_similarities = cosine_similarity(user_query_emb_c.reshape(1, -1), np.vstack(registry["code_embeddings"]))

# Add cosine similarity scores as a new column
registry_code=registry.copy()
registry_code["cosine_similarity_code_2"] = cos_similarities[0]

# Sort the dataframe based on cosine similarity
sorted_df_code = registry_code.sort_values(by="cosine_similarity_code_2", ascending=False)

# Retrieve the top 5 most similar documents
top_5_similar_code = sorted_df_code.head(5)
top_5_similar_code

,code,doc,doc_embeddings,code_embeddings,code_complete_embeddings,cosine_similarity_code_2
0,"def add(a, b):\n return a + b",This function adds two numbers.,"[tensor(-1.5903), tensor(-0.3973), tensor(3.99...","[-0.4316707, -0.029330524, 2.6694558, 3.79384,...","[0.31938395, -0.19940263, 0.23405677, -0.42716...",0.823493
2,"def multiply(a, b):\n return a * b",This function multiplies two numbers.,"[tensor(-2.3068), tensor(1.3498), tensor(3.078...","[-1.0338337, 1.418475, 1.0938523, 3.0212605, 0...","[0.28581232, -0.23162104, 0.2407104, -0.225170...",0.603547
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...,"[tensor(-2.4678), tensor(-2.2801), tensor(0.66...","[-1.266672, -0.6931733, -0.43216208, 2.7053928...","[0.2588312, -0.2687941, 0.3244955, 0.07137667,...",0.482005
5,"def modulus(a, b):\n return a % b",This function returns the remainder when the f...,"[tensor(-2.4903), tensor(-1.3584), tensor(1.60...","[-0.21913469, 0.7265679, -0.20258345, 1.877981...","[0.21118452, -0.15059435, 0.5316688, 0.0912161...",0.453946
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...,"[tensor(-3.0391), tensor(-0.0748), tensor(1.79...","[-0.93327546, 2.0973008, 0.7891953, 1.654753, ...","[0.34864905, -0.23415264, 0.2369114, 0.0368931...",0.436676


# Code Sumarization

In [ ]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base-multi-sum')


In [ ]:
def generate_summary(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    generated_ids = model.generate(input_ids, max_length=20)
    summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
registry_summary=registry.copy()
# Assuming the registry dataframe is already defined
registry_summary["summarization"] = registry["code"].apply(generate_summary)
registry_summary


,code,doc,doc_embeddings,code_embeddings,summarization
0,"def add(a, b):\n return a + b",This function adds two numbers.,"[-1.590325, -0.39731324, 3.997744, 2.6590736, ...","[2.1682317, -0.6963131, 1.542846, 3.1269562, 1...",Add two vectors.
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...,"[-2.4677699, -2.280132, 0.6681174, 1.4311584, ...","[0.2956554, 0.97152746, 0.78425086, 3.7197409,...",Subtract two vectors.
2,"def multiply(a, b):\n return a * b",This function multiplies two numbers.,"[-2.3068378, 1.3498034, 3.0785217, 1.3164998, ...","[3.8874378, -2.6447327, -0.012378594, 2.499000...",Multiply two vectors.
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...,"[-3.0390859, -0.07484988, 1.7925602, 0.2812554...","[2.7209783, 0.7028546, 1.585468, 1.0704353, 2....",Divide two numbers.
4,"def power(a, b):\n return a ** b",This function raises the first number to the p...,"[-2.946074, -0.51749504, 2.3797052, 2.3707116,...","[3.9257975, -2.2621982, 1.2599363, 4.382963, 0...",Returns a power of b.
5,"def modulus(a, b):\n return a % b",This function returns the remainder when the f...,"[-2.490343, -1.3584495, 1.6064605, -0.03004119...","[1.845908, 1.1119063, 1.2776934, 0.43036178, 2...",Returns a % b.
